In [3]:
from model import *

import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers
from pytorch_lightning import Trainer
import numpy as np

from net import SimpleNet
from datamodule import Datamodule
from model import FlightModel

from file_parsing_utils import create_csv_dict
from coordinate_transform import CoordinateEnum, helper_get_coordinate_system_based_on_enum

In [4]:
num_workers = 8

max_num_val_maps = 8
coordinate_system_enum = CoordinateEnum.LatLongCoordinates
coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_enum=coordinate_system_enum)

auxiliary_input_channels = [
                            "diff_time", 
                            "flight_course_corrected", 
                            "flight_course_unknown", 
                            ]

auxiliary_output_channels = []



num_res_blocks = 8
in_channels = len(coordinate_system) + len(auxiliary_input_channels)
out_channels = len(coordinate_system)
###dilation = np.concatenate([np.ones(1), np.random.randint(1, 5, num_res_blocks), np.ones(1)])
dilation = [1 if i % 2 == 0 else 2 for i in range(num_res_blocks + 2)]
net = SimpleNet(in_channels=in_channels, 
                out_channels=out_channels, 
                intermediate_channels=128, 
                num_res_blocks=num_res_blocks, 
                num_output_rows=1, 
                dilation = dilation, 
                kernel_size = 27, 
                use_bn_norm=False, #True, 
                stride=1, 
                bias=True)


individual_flights_dir = "/Users/aleksandranikevich/Desktop/AircraftTrajectory/data/Individual_Flights/"
flight_dfs = create_csv_dict(individual_flights_dir)


datamodule = Datamodule(all_flight_dataframes_dict = flight_dfs, 
                        num_input_rows_total = 1000, 
                        min_rows_input = 100, 
                        num_output_rows = 1, 
                        coordinate_system_enum = coordinate_system_enum,
                        auxiliary_input_channels = auxiliary_input_channels,
                        auxiliary_output_channels = auxiliary_output_channels,
                        train_prop = 0.8, 
                        batch_size = 32, 
                        num_workers = num_workers, 
                        pin_memory = True,)

train_dataloader = datamodule.train_dataloader()
test_dataloader = datamodule.test_dataloader()

# dummy tensor
test_dataloader_iterator = test_dataloader.__iter__()
tensors_dict = next(test_dataloader_iterator)
dummy_input_tensor = tensors_dict["input_tensor"]
dummy_output = net(dummy_input_tensor)

debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False
debug use batch norm:  False


KeyboardInterrupt: 

In [21]:
tensors_dict["input_tensor"][0, :2, :20]

tensor([[ 28.3660,  28.3660,  28.3660,  28.3660,  28.3660,  28.3660,  28.3660,
          28.3660,  28.3660,  28.3660,  28.3660,  28.3660,  28.3660,  28.3660,
          28.3660,  28.3660,  28.3660,  28.3660,  28.3660,  28.3660],
        [-90.2972, -90.2972, -90.2972, -90.2972, -90.2972, -90.2972, -90.2972,
         -90.2972, -90.2972, -90.2972, -90.2972, -90.2972, -90.2972, -90.2972,
         -90.2972, -90.2972, -90.2972, -90.2972, -90.2972, -90.2972]])

In [22]:
batch = tensors_dict

batch = normalize_center(batch, coordinate_system_enum=coordinate_system_enum)
input_tensor = batch["input_tensor"]
output_tensor = batch["output_tensor"]
list_full_flightpaths = batch["meta_flightpath"]
list_of_msn = batch["meta_msn"]
pred_tensor = net(input_tensor) # torch.Size([32, 4, 1])
b = input_tensor.shape[0]

In [23]:
n = batch['normalization_tensor'][0].clone()
orig = input_tensor[0, :2, -10:] + n
orig

tensor([[ 29.1860,  29.1899,  29.1935,  29.1970,  29.2007,  29.2041,  29.2073,
          29.2106,  29.2140,  29.2174],
        [-90.5450, -90.5462, -90.5472, -90.5483, -90.5493, -90.5504, -90.5514,
         -90.5523, -90.5534, -90.5544]])

In [24]:
pred_tensor.shape

torch.Size([32, 2, 1])

In [25]:
batch["input_tensor"][0, :2, :20]

tensor([[-0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514,
         -0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514, -0.8514,
         -0.8514, -0.8514, -0.8514, -0.8514],
        [ 0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,
          0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,  0.2572,
          0.2572,  0.2572,  0.2572,  0.2572]])

In [38]:
# def iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system):
#     with torch.no_grad():
#         _, _, num_input_rows = input_tensor.shape
#         _, _, num_output_rows = prediction_tensor.shape
#         input_tensor_final_n_rows = input_tensor[:, :, -num_output_rows:]  
#         stacked_orig_pred = torch.concat([input_tensor, input_tensor_final_n_rows], dim=-1) # keep auxilary info, pred tensor will then replace the coordinate values
#         stacked_orig_pred[:, :len_coordinate_system, -num_output_rows:] = prediction_tensor 
#         new_normalization_tensor = stacked_orig_pred[:, :len_coordinate_system, -1: ]

#         print("inside step: ", stacked_orig_pred[0, 0, -10:])
#         print("inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

#         stacked_orig_pred[:, :len_coordinate_system, :] = stacked_orig_pred[:, :len_coordinate_system, :] - new_normalization_tensor # new input tensor

#         print("Before end of WITH inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

#     print("BEFORE RETURN inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])
#     return torch.clone(stacked_orig_pred[:, :, -num_input_rows:]), torch.clone(new_normalization_tensor)

def iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system):
    with torch.no_grad():
        _, _, num_input_rows = input_tensor.shape
        _, _, num_output_rows = prediction_tensor.shape
        input_tensor_final_n_rows = input_tensor[:, :, -num_output_rows:]  
        stacked_orig_pred = torch.concat([input_tensor, input_tensor_final_n_rows], dim=-1) # keep auxiliary info, pred tensor will then replace the coordinate values
        stacked_orig_pred[:, :len_coordinate_system, -num_output_rows:] = prediction_tensor 
        new_normalization_tensor = stacked_orig_pred[:, :len_coordinate_system, -1:].clone()  # Clone to avoid modifying the original tensor

        print("inside step: ", stacked_orig_pred[0, 0, -10:])
        print("inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

        stacked_orig_pred[:, :len_coordinate_system, :] = stacked_orig_pred[:, :len_coordinate_system, :] - new_normalization_tensor # new input tensor

        print("Before end of WITH inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

    print("BEFORE RETURN inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])
    return torch.clone(stacked_orig_pred[:, :, -num_input_rows:]), torch.clone(new_normalization_tensor)



def iterative_path_predict(batch_dict, model, coordinate_system_enum, num_predict_steps, bool_normalize_center = False):
    """
    iteratively predict N steps using the model
    """
    
    coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_system_enum)
    len_coordinate_system = len(coordinate_system )

    if bool_normalize_center:
        batch_dict = normalize_center(batch_dict, coordinate_system_enum=coordinate_system_enum)

    input_tensor = batch_dict["input_tensor"]
    normalization_tensor_cumulative = torch.clone(batch_dict["normalization_tensor"])
    print("at the beginning normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
    # prediction_tensor = model(input_tensor)
    # print("prediction_tensor   : ", prediction_tensor[0, 0, :10])

    pred_tensors_list = []

    for i in range(num_predict_steps):
        prediction_tensor = model(input_tensor)
        input_tensor, normalization_tensor = iterative_path_predict_step(input_tensor.clone(), prediction_tensor.clone(), coordinate_system_enum, len_coordinate_system)
        print("prediction_tensor: ", prediction_tensor[0, 0, :10])
        print("normalization_tensor: ", normalization_tensor[0, 0])
        print("new input_tensor: ", input_tensor[0, 0, -10:])
        #######normalization_tensor_cumulative = normalization_tensor_cumulative + normalization_tensor.clone()
        pred_tensors_list.append(prediction_tensor.clone())

        print("")

    
    iterative_predictions_tensor = torch.concat(pred_tensors_list, dim=-1) # with normalization: normalization_tensor_copy
    print("before CUMSUM PRED TENSOR: ", iterative_predictions_tensor[0])
    iterative_predictions_tensor = torch.cumsum(iterative_predictions_tensor, dim=-1) #np.cumsum(iterative_predictions_tensor, axis=-1)
    print("at the end iterative_predictions_tensor: ", iterative_predictions_tensor[0])
    print("at the end normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
    iterative_predictions_tensor = un_normalize_center_predictions(iterative_predictions_tensor, normalization_tensor_cumulative) 
    iterative_predictions_tensor_np = iterative_predictions_tensor.detach().cpu().numpy()
    return iterative_predictions_tensor_np


# def iterative_path_predict(batch_dict, model, coordinate_system_enum, num_predict_steps, bool_normalize_center = False):
#     """
#     iteratively predict N steps using the model
#     """
    
#     coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_system_enum)
#     len_coordinate_system = len(coordinate_system )

#     if bool_normalize_center:
#         batch_dict = normalize_center(batch_dict, coordinate_system_enum=coordinate_system_enum)

#     input_tensor = batch_dict["input_tensor"]
#     normalization_tensor_cumulative = torch.clone(batch_dict["normalization_tensor"])
#     print("at the beginning normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
#     prediction_tensor = model(input_tensor)
#     print("prediction_tensor   : ", prediction_tensor[0, 0, :10])

#     pred_tensors_list = [torch.clone(prediction_tensor)] 

#     for i in range(num_predict_steps):
#         input_tensor, normalization_tensor = iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system)
#         prediction_tensor = model(input_tensor)
#         print("prediction_tensor: ", prediction_tensor[0, 0, :10])
#         print("normalization_tensor: ", normalization_tensor[0, 0])
#         print("new input_tensor: ", input_tensor[0, 0, -10:])
#         normalization_tensor_cumulative = normalization_tensor_cumulative + normalization_tensor
#         pred_tensors_list.append(prediction_tensor)

#         print("")

    
#     iterative_predictions_tensor = torch.concat(pred_tensors_list, dim=-1) # with normalization: normalization_tensor_copy
#     print("before CUMSUM PRED TENSOR: ", iterative_predictions_tensor[0])
#     iterative_predictions_tensor = torch.cumsum(iterative_predictions_tensor, dim=-1) #np.cumsum(iterative_predictions_tensor, axis=-1)
#     print("at the end iterative_predictions_tensor: ", iterative_predictions_tensor[0])
#     print("at the end normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
#     iterative_predictions_tensor = un_normalize_center_predictions(iterative_predictions_tensor, normalization_tensor_cumulative) 
#     iterative_predictions_tensor_np = iterative_predictions_tensor.detach().cpu().numpy()
#     return iterative_predictions_tensor_np



def dummy_net(dummy_arg):
    return 0.01 * torch.ones((32, 2, 1))


In [57]:
import torch

def dummy_net2(dummy_arg):
    # Create a tensor with the desired values for timesteps
    timestep_values = torch.tensor([0.01, 0.02, 0.03])
    
    # Repeat the tensor for each batch and channel
    dummy_tensor = timestep_values.repeat(32, 2, 1)
    
    return dummy_tensor

# # Example usage
# dummy_arg = None  # Replace with actual argument if needed
# output = dummy_net2(dummy_arg)
# print(output.shape)
# print(output[0, 0])


In [58]:
n_future_timesteps = 10
iterative_predictions_tensor_np = iterative_path_predict(batch, dummy_net2, coordinate_system_enum, num_predict_steps=n_future_timesteps, bool_normalize_center = False)
iterative_predictions_tensor_np.shape

at the beginning normalization_tensor_cumulative:  tensor([29.2174])
inside step:  tensor([-0.0204, -0.0168, -0.0134, -0.0101, -0.0068, -0.0034,  0.0000,  0.0100,
         0.0200,  0.0300])
inside step new_normalization_tensor:  tensor([0.0300])
Before end of WITH inside step new_normalization_tensor:  tensor([0.0300])
BEFORE RETURN inside step new_normalization_tensor:  tensor([0.0300])
prediction_tensor:  tensor([0.0100, 0.0200, 0.0300])
normalization_tensor:  tensor([0.0300])
new input_tensor:  tensor([-0.0504, -0.0468, -0.0434, -0.0401, -0.0368, -0.0334, -0.0300, -0.0200,
        -0.0100,  0.0000])

inside step:  tensor([-0.0401, -0.0368, -0.0334, -0.0300, -0.0200, -0.0100,  0.0000,  0.0100,
         0.0200,  0.0300])
inside step new_normalization_tensor:  tensor([0.0300])
Before end of WITH inside step new_normalization_tensor:  tensor([0.0300])
BEFORE RETURN inside step new_normalization_tensor:  tensor([0.0300])
prediction_tensor:  tensor([0.0100, 0.0200, 0.0300])
normalization_

(32, 2, 30)

In [59]:
33.7337 - 33.8337


-0.10000000000000142

In [60]:
iterative_predictions_tensor_np[0, 0]

array([29.22743 , 29.24743 , 29.27743 , 29.28743 , 29.30743 , 29.337431,
       29.34743 , 29.36743 , 29.39743 , 29.40743 , 29.42743 , 29.45743 ,
       29.46743 , 29.48743 , 29.51743 , 29.52743 , 29.54743 , 29.57743 ,
       29.587431, 29.60743 , 29.63743 , 29.64743 , 29.66743 , 29.69743 ,
       29.70743 , 29.72743 , 29.757431, 29.76743 , 29.78743 , 29.81743 ],
      dtype=float32)

In [61]:
iterative_predictions_tensor_np.shape

(32, 2, 30)

In [62]:
n = batch['normalization_tensor'][0].clone()
orig = input_tensor[0, :2, -10:] + n
orig



tensor([[ 29.1860,  29.1899,  29.1935,  29.1970,  29.2007,  29.2041,  29.2073,
          29.2106,  29.2140,  29.2174],
        [-90.5450, -90.5462, -90.5472, -90.5483, -90.5493, -90.5504, -90.5514,
         -90.5523, -90.5534, -90.5544]])

In [63]:
print(iterative_predictions_tensor_np.shape)
iterative_predictions_tensor_np[0]

(32, 2, 30)


array([[ 29.22743 ,  29.24743 ,  29.27743 ,  29.28743 ,  29.30743 ,
         29.337431,  29.34743 ,  29.36743 ,  29.39743 ,  29.40743 ,
         29.42743 ,  29.45743 ,  29.46743 ,  29.48743 ,  29.51743 ,
         29.52743 ,  29.54743 ,  29.57743 ,  29.587431,  29.60743 ,
         29.63743 ,  29.64743 ,  29.66743 ,  29.69743 ,  29.70743 ,
         29.72743 ,  29.757431,  29.76743 ,  29.78743 ,  29.81743 ],
       [-90.544426, -90.52443 , -90.49443 , -90.48443 , -90.46443 ,
        -90.434425, -90.42443 , -90.40443 , -90.37443 , -90.364426,
        -90.34443 , -90.31443 , -90.30443 , -90.28443 , -90.254425,
        -90.24443 , -90.22443 , -90.19443 , -90.184425, -90.16443 ,
        -90.13443 , -90.12443 , -90.10443 , -90.074425, -90.06443 ,
        -90.044426, -90.01443 , -90.004425, -89.98443 , -89.95443 ]],
      dtype=float32)

In [67]:
29.2174 - 29.22743

-0.010029999999996875

In [65]:
-105.8839 + 105.77392

-0.10997999999999308

In [66]:
lat = iterative_predictions_tensor_np[0, 0]
long = iterative_predictions_tensor_np[0, 1]
min_lat, max_lat = np.min(lat), np.amax(lat)
min_long, max_long = np.min(long), np.amax(long)
m = create_folium_map(min_lat, min_long, max_lat, max_long, border_lat_prop=0.2, border_long_prop=0.15, tiles=None)

folium.PolyLine(locations=iterative_predictions_tensor_np[0].T, color='red', weight=2.5, opacity=1).add_to(m)
folium.PolyLine(locations=orig.T, color='black', weight=2.5, opacity=1).add_to(m)

m

# m = create_folium_map(min_lat, min_long, max_lat, max_long, border_lat_prop=0.2, border_long_prop=0.15, tiles=None) #"Cartodb dark_matter")
# folium.PolyLine(locations=flightpath_compleate, color='black', weight=2.5, opacity=1).add_to(m)

# for predicted_flightpath in predicted_paths_list:
#     folium.PolyLine(locations=predicted_flightpath, color='blue', weight=2.5, opacity=1).add_to(m)



# m

In [86]:
# def iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system):
#     with torch.no_grad():
#         _, _, num_input_rows = input_tensor.shape
#         _, _, num_output_rows = prediction_tensor.shape
#         input_tensor_final_n_rows = input_tensor[:, :, -num_output_rows:]  
#         stacked_orig_pred = torch.concat([input_tensor, input_tensor_final_n_rows], dim=-1) # keep auxilary info, pred tensor will then replace the coordinate values
#         stacked_orig_pred[:, :len_coordinate_system, -num_output_rows:] = prediction_tensor 
#         new_normalization_tensor = stacked_orig_pred[:, :len_coordinate_system, -1: ]

#         print("inside step: ", stacked_orig_pred[0, 0, -10:])
#         print("inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

#         stacked_orig_pred[:, :len_coordinate_system, :] = stacked_orig_pred[:, :len_coordinate_system, :] - new_normalization_tensor # new input tensor

#         print("Before end of WITH inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

#     print("BEFORE RETURN inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])
#     return torch.clone(stacked_orig_pred[:, :, -num_input_rows:]), torch.clone(new_normalization_tensor)

def iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system):
    with torch.no_grad():
        _, _, num_input_rows = input_tensor.shape
        _, _, num_output_rows = prediction_tensor.shape
        input_tensor_final_n_rows = input_tensor[:, :, -num_output_rows:]  
        stacked_orig_pred = torch.concat([input_tensor, input_tensor_final_n_rows], dim=-1) # keep auxiliary info, pred tensor will then replace the coordinate values
        stacked_orig_pred[:, :len_coordinate_system, -num_output_rows:] = prediction_tensor 
        new_normalization_tensor = stacked_orig_pred[:, :len_coordinate_system, -1:].clone()  # Clone to avoid modifying the original tensor

        #print("inside step: ", stacked_orig_pred[0, 0, -10:])
        #print("inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

        stacked_orig_pred[:, :len_coordinate_system, :] = stacked_orig_pred[:, :len_coordinate_system, :] - new_normalization_tensor # new input tensor

        #print("Before end of WITH inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])

    #print("BEFORE RETURN inside step new_normalization_tensor: ", new_normalization_tensor[0, 0])
    return torch.clone(stacked_orig_pred[:, :, -num_input_rows:]), torch.clone(new_normalization_tensor)




def iterative_path_predict(batch_dict, model, coordinate_system_enum, num_predict_steps, bool_normalize_center = False):
    """
    iteratively predict N steps using the model
    """
    
    coordinate_system = helper_get_coordinate_system_based_on_enum(coordinate_system_enum)
    len_coordinate_system = len(coordinate_system )

    if bool_normalize_center:
        batch_dict = normalize_center(batch_dict, coordinate_system_enum=coordinate_system_enum)

    input_tensor = batch_dict["input_tensor"]
    normalization_tensor_cumulative = torch.clone(batch_dict["normalization_tensor"])
    print("at the beginning normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
    prediction_tensor = model(input_tensor)
    print("prediction_tensor   : ", prediction_tensor[0, 0, :10])

    pred_tensors_list = [torch.clone(prediction_tensor)] 

    for i in range(num_predict_steps):
        input_tensor, normalization_tensor = iterative_path_predict_step(input_tensor, prediction_tensor, coordinate_system_enum, len_coordinate_system)
        prediction_tensor = model(input_tensor)
        #print("prediction_tensor: ", prediction_tensor[0, 0, :10])
        #print("normalization_tensor: ", normalization_tensor[0, 0])
        #print("new input_tensor: ", input_tensor[0, 0, -10:])
        normalization_tensor_cumulative = normalization_tensor_cumulative + normalization_tensor
        pred_tensors_list.append(prediction_tensor)

        print("normalization_tensor_cumulative: ", normalization_tensor_cumulative[0])

    
    iterative_predictions_tensor = torch.concat(pred_tensors_list, dim=-1) # with normalization: normalization_tensor_copy
    print("at the end iterative_predictions_tensor: ", iterative_predictions_tensor[0, 0, -10:])
    print("at the end normalization_tensor_cumulative: ", normalization_tensor_cumulative[0, 0])
    # iterative_predictions_tensor = un_normalize_center_predictions(iterative_predictions_tensor, normalization_tensor_cumulative) 
    # iterative_predictions_tensor_np = iterative_predictions_tensor.detach().cpu().numpy()
    # return iterative_predictions_tensor_np

    return iterative_predictions_tensor, normalization_tensor_cumulative



def dummy_net(dummy_arg):
    return 0.01 * torch.ones((32, 2, 1))


In [87]:
n_future_timesteps = 10
iterative_predictions_tensor, normalization_tensor_cumulative = iterative_path_predict(batch, dummy_net, coordinate_system_enum, num_predict_steps=n_future_timesteps, bool_normalize_center = False)


at the beginning normalization_tensor_cumulative:  tensor([39.3563])
prediction_tensor   :  tensor([0.0100])
normalization_tensor_cumulative:  tensor([[  39.3663],
        [-105.8739]])
normalization_tensor_cumulative:  tensor([[  39.3763],
        [-105.8639]])
normalization_tensor_cumulative:  tensor([[  39.3863],
        [-105.8539]])
normalization_tensor_cumulative:  tensor([[  39.3963],
        [-105.8439]])
normalization_tensor_cumulative:  tensor([[  39.4063],
        [-105.8339]])
normalization_tensor_cumulative:  tensor([[  39.4163],
        [-105.8239]])
normalization_tensor_cumulative:  tensor([[  39.4263],
        [-105.8139]])
normalization_tensor_cumulative:  tensor([[  39.4363],
        [-105.8039]])
normalization_tensor_cumulative:  tensor([[  39.4463],
        [-105.7939]])
normalization_tensor_cumulative:  tensor([[  39.4563],
        [-105.7839]])
at the end iterative_predictions_tensor:  tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,

In [84]:
iterative_predictions_tensor

tensor([[[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
          0.0100, 0.0100, 0.0100],
         [0

In [85]:
normalization_tensor_cumulative

tensor([[[  39.4563],
         [-105.7839]],

        [[  35.6005],
         [-110.6333]],

        [[  43.9342],
         [ -85.1787]],

        [[  33.7318],
         [ -84.2669]],

        [[  33.4189],
         [ -93.9903]],

        [[  32.6742],
         [-116.8587]],

        [[  33.7202],
         [-111.5278]],

        [[  42.8627],
         [ -87.3010]],

        [[  42.0674],
         [ -87.7974]],

        [[  34.2349],
         [-112.0497]],

        [[  33.7495],
         [ -84.3348]],

        [[  39.7617],
         [ -77.9656]],

        [[  40.8877],
         [-111.8768]],

        [[  34.8683],
         [-116.3792]],

        [[  30.4466],
         [ -95.6123]],

        [[  39.0388],
         [-104.3287]],

        [[  40.8908],
         [ -73.8944]],

        [[  43.3235],
         [ -87.9746]],

        [[  33.7478],
         [ -84.3414]],

        [[  30.0928],
         [ -90.1655]],

        [[  35.7002],
         [ -84.1626]],

        [[  33.5313],
         [-1

In [91]:
np.cumsum(iterative_predictions_tensor, axis=-1)[0, 0]

tensor([0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800, 0.0900,
        0.1000, 0.1100])

In [88]:
iterative_predictions_tensor.shape

torch.Size([32, 2, 11])

In [77]:
iterative_predictions_tensor_np[0, :, :]

array([[  39.466293,   39.466293,   39.466293,   39.466293,   39.466293,
          39.466293,   39.466293,   39.466293,   39.466293,   39.466293,
          39.466293],
       [-105.77392 , -105.77392 , -105.77392 , -105.77392 , -105.77392 ,
        -105.77392 , -105.77392 , -105.77392 , -105.77392 , -105.77392 ,
        -105.77392 ]], dtype=float32)

In [78]:
np.diff(iterative_predictions_tensor_np[0, 0, :])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [32]:
np.min(iterative_predictions_tensor_np)

-122.3798

In [33]:
# Find the value closest to zero
closest_to_zero = iterative_predictions_tensor_np.flat[np.abs(iterative_predictions_tensor_np).argmin()]

print("Value closest to zero:", closest_to_zero)

Value closest to zero: 29.99376


In [34]:
iterative_predictions_tensor_np[0, :, :].shape

(2, 11)

In [140]:
ones = torch.ones(2, 10)*0.01
ones

tensor([[0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
         0.0100],
        [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
         0.0100]])

In [142]:
torch.cumsum(ones, dim=-1)

tensor([[0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800, 0.0900,
         0.1000],
        [0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800, 0.0900,
         0.1000]])